# Higgs Twitter Dataset

This notebook documents the **Higgs Twitter dataset** from the SNAP collection.

It shows:
- where the data is stored on the DezInfo server
- how it is accessed via WSL + sshfs
- the structure of each file
- the first rows ("head") of each dataset

Source:
https://snap.stanford.edu/data/higgs-twitter.html